In [ ]:
from google.colab import drive
drive.mount("/content/drive")

path_final = "/content/drive/MyDrive/LTSSUD"
import os

if (os.path.isdir(path_final) == True):
  %cd "/content/drive/MyDrive/LTSSUD"

!ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/LTSSUD
Data_Preparation___Image_processing.ipynb  sequence.ipynb
dataset					   test.csv
img_100.npz				   train.csv
img_28.npz				   train_data_128_full.npz
models					   Untitled0.ipynb
parallel.ipynb				   winequality-red.csv
Proposal.gdoc


In [ ]:
import numpy as np
import pandas as pd
import math
import matplotlib as plt
import plotly.express as px
import random

In [ ]:
train = np.load('dataset/train_data_128_full.npz',allow_pickle=True)
train_X = train['data']
train_y = train['label']
train_p = train['plant']

In [ ]:
classes, counts = np.unique(train_y, return_counts=True)

In [ ]:
fig = px.line(x=classes, y= np.full(len(classes), np.mean(counts)))
fig.add_bar(x= classes, y = counts)
fig.show()

In [ ]:
delta = []
for i in classes:
  t = np.ones(len(counts))
  t[i] = 0
  delta.append(np.mean(counts) - counts[i])

In [ ]:
print(delta)

[639.5263157894738, 646.5263157894738, 923.5263157894738, -172.47368421052624, -58.473684210526244, 301.52631578947376, 459.52631578947376, 732.5263157894738, 189.52631578947376, 355.52631578947376, 213.52631578947376, 199.52631578947376, 36.526315789473756, 282.52631578947376, 804.5263157894738, -3262.4736842105262, -694.4736842105262, 855.5263157894738, 345.52631578947376, -38.473684210526244, 343.52631578947376, 343.52631578947376, 1021.5263157894738, 846.5263157894738, -2928.4736842105262, -324.47368421052624, 255.52631578947376, 778.5263157894738, -558.4736842105262, 343.52631578947376, -383.47368421052624, 381.52631578947376, -273.47368421052624, -197.47368421052624, 19.526315789473756, -3142.4736842105262, 844.5263157894738, -129.47368421052624]


In [ ]:
delta = np.array(delta)
idx = (np.absolute(delta) > np.quantile(np.absolute(delta), 0.65))
print(np.quantile(np.absolute(delta), 0.65))

639.8763157894738


In [ ]:
fig = px.bar(x= classes, y = counts, color = idx)
fig.show()

In [ ]:
classes[idx].astype(int)

array([ 1,  2,  7, 14, 15, 16, 17, 22, 23, 24, 27, 35, 36])

In [ ]:
data = np.concatenate((train_X, train_p.reshape(1,-1).T), axis=1)
data = np.concatenate((data, train_y.reshape(1,-1).T), axis=1)
t_data = np.copy(data)

In [ ]:
del_idx = np.zeros(len(data))

for i in classes[idx].astype(int):
  if(round(delta[i]) >= 0):
    t = data[data[:,-1] == i]
    r_arr = np.random.randint(t.shape[0], size = abs(round(delta[i])))
    t_data = np.concatenate((t_data, t[r_arr, :]), axis=0)

del_idx = np.zeros(len(t_data))

for i in classes[idx].astype(int):
  if(round(delta[i]) < 0):
    idt = np.where(data[:,-1] == i)
    r_arr = np.arange(idt[0].shape[0])
    np.random.shuffle(r_arr)
    r_arr = r_arr[0:abs(round(delta[i]))]
    #print(r_arr)
    del_idx[idt[0][r_arr]] = 1

t_data = t_data[~del_idx.astype(bool),:]
np.random.shuffle(t_data)

In [ ]:
data.shape[0]

43454

In [ ]:
t_data.shape[0]

40886

In [ ]:
train_X = t_data[:, 0:-2]
train_y = t_data[:, -1]
train_p = t_data[:, -2]

In [ ]:
classes, counts = np.unique(train_y, return_counts=True)

In [ ]:
fig = px.line(x=classes, y= np.full(len(classes), np.mean(counts)))
fig.add_bar(x= classes, y = counts)
fig.show()

In [ ]:
np.savez('train_data_128_full.npz', data = train_X, label = train_y, plant = train_p)